# Exploratory Data Anaysis of COVID-19

In [4]:
import pandas as pd

In [5]:
df=pd.read_csv('covid-19.csv',delimiter=',')

## Understand the data
     Data distruibution 
						- df.shape , df.head(5) -- get to know data distribution
						- df.describe()
						- df.info(memory_usage = 'deep')
    
    

In [6]:
df.head(5) 

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Admin2.1,Province_State.1,Country_Region.1
0,45001.0,Abbeville,South Carolina,US,2020-05-06 02:32:31,34.22333378,-82.461707,33.0,0,0,33,Abbeville,South Carolina,US,NaN
1,22001.0,Acadia,Louisiana,US,2020-05-06 02:32:31,30.295064899999996,-92.414197,136.0,10,0,126,Acadia,Louisiana,US,NaN
2,51001.0,Accomack,Virginia,US,2020-05-06 02:32:31,37.76707161,-75.632346,429.0,7,0,422,Accomack,Virginia,US,NaN
3,16001.0,Ada,Idaho,US,2020-05-06 02:32:31,43.4526575,-116.241552,713.0,19,0,694,Ada,Idaho,US,NaN
4,19001.0,Adair,Iowa,US,2020-05-06 02:32:31,41.33075609,-94.471059,2.0,0,0,2,Adair,Iowa,US,NaN


In [11]:
print('*'*100)
print('Total no of rows and columns are {},{} respectively'.format(df.shape[0],df.shape[1]))
print('Dataframe information is :')
print(df.info(memory_usage = 'deep'))
print('*'*100)

****************************************************************************************************
Total no of rows and columns are 3208,15 respectively
Dataframe information is :
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3208 entries, 0 to 3207
Data columns (total 15 columns):
FIPS                2934 non-null float64
Admin2              2937 non-null object
Province_State      3025 non-null object
Country_Region      3208 non-null object
Last_Update         3208 non-null object
Lat                 3142 non-null object
Long_               3142 non-null float64
Confirmed           3208 non-null float64
Deaths              3208 non-null int64
Recovered           3208 non-null int64
Active              3208 non-null int64
Combined_Key        3208 non-null object
Admin2.1            3026 non-null object
Province_State.1    2938 non-null object
Country_Region.1    0 non-null float64
dtypes: float64(4), int64(3), object(8)
memory usage: 1.7 MB
None
********************************

In [12]:
df.dtypes

FIPS                float64
Admin2               object
Province_State       object
Country_Region       object
Last_Update          object
Lat                  object
Long_               float64
Confirmed           float64
Deaths                int64
Recovered             int64
Active                int64
Combined_Key         object
Admin2.1             object
Province_State.1     object
Country_Region.1    float64
dtype: object

## Data Preprocessing - preparing data for Data Analysis 
    - Data formatting (columns and its types, data values)
    - Identify missing values and handling it
    - Data binning
    - Transaformation

In [13]:
# drop extra columns
#Combined_Key	Admin2.1	Province_State.1	Country_Region.1 as they are repetitive columns
df.drop(columns=['FIPS','Combined_Key','Admin2.1','Province_State.1','Country_Region.1'],inplace=True)

In [14]:
#Rename Admin2-->Admin and long_ --> Long
df.rename(columns={'Admin2':'Admin','Long_': 'Long'},inplace=True)

In [17]:
df.dtypes #change confirmed cases datatype to int64 from float64 as cases cant be in decimals.
df['Confirmed']=df['Confirmed'].astype('int64')

In [45]:
# 1) Manually handle null values
# Identify and handle missing values
#df['Admin'].fillna('Admin',inplace=True)  # fill Nan of Admin to default Admin
#df.dropna(subset=['Province_State'],inplace=True) # drop Nan values

# 2) automatically idenitify columns with null values and replace null with Mode/Average/user-defined (NA)

# List of nullable columns
nullable_columns = []

check_null = df.isnull().sum()

for col in df.columns:
    if check_null[col]>0:
        nullable_columns.append(col)

#print(nullable_columns) 

# Find mode of nullable columns  
nullable_columns_mode ={}
for col in nullable_columns:
    nullable_columns_mode[col] = df[col].mode()[0]
#print(nullable_columns_mode)

# Replace null values with their corresponding mode value
for k,v in nullable_columns_mode.items():
    df[k].fillna(v,inplace=True)

#df.isnull().sum()     verify for any still null values

Admin             0
Province_State    0
Country_Region    0
Last_Update       0
Lat               0
Long              0
Confirmed         0
Deaths            0
Recovered         0
Active            0
dtype: int64

In [46]:
#Data bining
def critical_level(n):
    if n>1000:
        return 'L1'
    elif n<=1000 and n>=500:
        return 'L2'
    else:
        return 'L3'

    
#add new column critical_level and add criticality into it
df['critical_level']=df['Active'].apply(critical_level)
df

,Admin,Province_State,Country_Region,Last_Update,Lat,Long,Confirmed,Deaths,Recovered,Active,critical_level
0,Abbeville,South Carolina,US,2020-05-06 02:32:31,34.22333378,-82.461707,33,0,0,33,L3
1,Acadia,Louisiana,US,2020-05-06 02:32:31,30.295064899999996,-92.414197,136,10,0,126,L3
2,Accomack,Virginia,US,2020-05-06 02:32:31,37.76707161,-75.632346,429,7,0,422,L3
3,Ada,Idaho,US,2020-05-06 02:32:31,43.4526575,-116.241552,713,19,0,694,L2
4,Adair,Iowa,US,2020-05-06 02:32:31,41.33075609,-94.471059,2,0,0,2,L3
...,...,...,...,...,...,...,...,...,...,...,...
3203,Unassigned,Texas,West Bank and Gaza,2020-05-06 02:32:31,31.9522,35.233200,371,2,127,242,L3
3204,Unassigned,Texas,Western Sahara,2020-05-06 02:32:31,24.2155,-12.885800,6,0,5,1,L3
3205,Unassigned,Texas,Yemen,2020-05-06 02:32:31,15.552726999999999,48.516388,22,4,1,17,L3
3206,Unassigned,Texas,Zambia,2020-05-06 02:32:31,-13.133897,27.849332,138,3,92,43,L3


In [33]:
#Transformation
#change country case to upper
def upper_case(text):
    return text.upper()

df['Country_Region']=df['Country_Region'].apply(upper_case)
df

# another way is
#df['Province_State']=df['Province_State'].str.upper()
#df

,Admin,Province_State,Country_Region,Last_Update,Lat,Long,Confirmed,Deaths,Recovered,Active,critical_level
0,Abbeville,South Carolina,US,2020-05-06 02:32:31,34.22333378,-82.461707,33,0,0,33,L3
1,Acadia,Louisiana,US,2020-05-06 02:32:31,30.295064899999996,-92.414197,136,10,0,126,L3
2,Accomack,Virginia,US,2020-05-06 02:32:31,37.76707161,-75.632346,429,7,0,422,L3
3,Ada,Idaho,US,2020-05-06 02:32:31,43.4526575,-116.241552,713,19,0,694,L2
4,Adair,Iowa,US,2020-05-06 02:32:31,41.33075609,-94.471059,2,0,0,2,L3
...,...,...,...,...,...,...,...,...,...,...,...
3020,Admin,Western Australia,AUSTRALIA,2020-05-06 02:32:31,-31.9505,115.860500,551,9,528,14,L3
3021,Admin,Xinjiang,CHINA,2020-05-06 02:32:31,41.1129,85.240100,76,3,73,0,L3
3022,Admin,Yukon,CANADA,2020-05-06 02:32:31,64.2823,-135.000000,11,0,0,11,L3
3023,Admin,Yunnan,CHINA,2020-05-06 02:32:31,24.974,101.487000,185,2,181,2,L3


# *********************************************EDA***************************************************

##  Number of confirmed,death and active cases of each country in desc order


In [34]:
df.groupby(by=['Country_Region'])['Confirmed','Deaths','Active'].sum().sort_values(['Confirmed'],ascending=False)

,Confirmed,Deaths,Active
Country_Region,,,
US,1204351,71064,972526
CHINA,83968,4637,461
CANADA,63215,4190,32019
AUSTRALIA,6875,97,803
FRANCE,1752,39,674
UNITED KINGDOM,1253,74,253
DENMARK,198,0,2
NETHERLANDS,193,17,37


## Country where most cases recovered

In [35]:
df.groupby(by=['Country_Region'])['Recovered'].max().sort_values(ascending=False).head(1)

Country_Region
US    189791
Name: Recovered, dtype: int64

## country with minimum cases  recovered

In [36]:
df.groupby(by=['Country_Region'])['Recovered'].min().sort_values(ascending=True).head(1)

Country_Region
CANADA    0
Name: Recovered, dtype: int64